# Adjusting Revenue for Inflation #

In [1]:
import numpy as np
import pandas as pd
import json

import statsmodels.api as sm

import matplotlib.pyplot as plt 
%matplotlib inline

import random as rand
import seaborn as sns

from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from datetime import datetime

In [2]:
movies = pd.read_csv("cleaned_data.csv")

In [3]:
#movies["revenue"]

In [44]:
#Getting release dates
release_dates = movies.iloc[:,[0,14]]
release_dates
#type(release_dates)

,id,release_date
0,10003,1997-04-03
1,100042,2014-11-12
2,10012,2005-02-25
3,10013,1986-10-05
4,10014,1985-10-31
...,...,...
4508,9986,2006-12-15
4509,99861,2015-04-22
4510,9988,2006-08-25
4511,9989,2001-01-12


In [5]:
#Only Getting Data w/o NaN
release_dates_filt = release_dates[release_dates['release_date'].notnull()]

In [6]:
#Only Getting Data w/o NaN
#release_dates_f = release_dates[release_dates['release_date'].notnull()]

In [7]:
#Trying an example first
release_date_ex = release_dates_filt.iloc[0][1]
#print(release_date_ex)
#type(release_date_ex)

#Str to date format
#https://www.digitalocean.com/community/tutorials/python-string-to-datetime-strptime
datetime.strptime(release_date_ex, '%Y-%m-%d').date()

datetime.date(1997, 4, 3)

In [8]:
#Replace NaN data with 0
#release_dates.replace('nan','0000-00-00')
#release_dates.fillna('0000-00-00')

In [9]:
#Release dates transformed into DateTime format in a Pandas Dataset:

dates = []

for i in range(len(release_dates_filt)):
    #print(type(release_dates.iloc[i][1]))
    entry_date = datetime.strptime(release_dates_filt.iloc[i][1], '%Y-%m-%d').date()
    dates.append([release_dates_filt.iloc[i][0],entry_date])

dates = pd.DataFrame(dates)

In [10]:
print(type(dates.iloc[0][1]))
dates

<class 'datetime.date'>


,0,1
0,10003,1997-04-03
1,100042,2014-11-12
2,10012,2005-02-25
3,10013,1986-10-05
4,10014,1985-10-31
...,...,...
4508,9986,2006-12-15
4509,99861,2015-04-22
4510,9988,2006-08-25
4511,9989,2001-01-12


In [11]:
years = []
for i in range(len(release_dates_filt)):
    entry_year = dates.iloc[i][1].year
    years.append([release_dates_filt.iloc[i][0],entry_year])
years

[[10003, 1997],
 [100042, 2014],
 [10012, 2005],
 [10013, 1986],
 [10014, 1985],
 [10015, 1986],
 [10016, 2001],
 [10017, 1986],
 [10019, 1987],
 [10020, 1991],
 [10021, 1987],
 [10022, 2005],
 [10023, 1987],
 [10024, 2009],
 [100241, 2014],
 [10025, 2006],
 [10027, 2005],
 [10028, 2003],
 [10029, 1998],
 [10030, 2007],
 [100402, 2014],
 [10048, 2005],
 [10052, 2002],
 [10053, 2006],
 [10054, 2001],
 [10057, 1993],
 [10060, 2005],
 [10061, 1996],
 [10065, 2005],
 [10066, 2005],
 [10067, 2006],
 [10068, 1986],
 [10069, 2006],
 [10072, 1987],
 [10073, 2006],
 [10077, 2005],
 [100791, 2012],
 [10083, 1987],
 [10090, 2007],
 [10092, 2005],
 [10093, 2006],
 [10096, 2004],
 [10098, 1921],
 [101, 1994],
 [10110, 1987],
 [10112, 1970],
 [10117, 1988],
 [101173, 2011],
 [10126, 1988],
 [101267, 2012],
 [10127, 1988],
 [10128, 1988],
 [101299, 2013],
 [10131, 1988],
 [10132, 1988],
 [10134, 1989],
 [10135, 1989],
 [10136, 1986],
 [10137, 1999],
 [10138, 2010],
 [10139, 2008],
 [10140, 2010],
 [1

In [12]:
years_arr = np.array(years)
years_arr = years_arr.T

In [13]:
years_df = pd.DataFrame()
years_df["id"] = years_arr[0]
years_df["Year"] = years_arr[1]
years_df

,id,Year
0,10003,1997
1,100042,2014
2,10012,2005
3,10013,1986
4,10014,1985
...,...,...
4508,9986,2006
4509,99861,2015
4510,9988,2006
4511,9989,2001


In [14]:
# We need inflation data or ticket data from 1915~current
# Everything will be converted into estimated number of tickets sold, then back to 2023 rates 

ticketprices = pd.read_csv("yearly_ticketprice.csv")
ticketprices
#ticketprices.dtypes

,Ticket Price,Year
0,0.07,1910
1,0.07,1911
2,0.07,1912
3,0.07,1913
4,0.07,1914
...,...,...
109,9.16,2019
110,9.41,2020
111,10.40,2021
112,10.45,2022


In [15]:
final_tx = years_df.join(ticketprices.set_index("Year"),on="Year")
final_tx

,id,Year,Ticket Price
0,10003,1997,4.59
1,100042,2014,8.17
2,10012,2005,6.41
3,10013,1986,3.71
4,10014,1985,3.55
...,...,...,...
4508,9986,2006,6.55
4509,99861,2015,8.43
4510,9988,2006,6.55
4511,9989,2001,5.66


In [16]:
#final_tx[(final_tx["Year"] == 2015)]
#final_tx.iloc[:,[1,2]]

In [17]:
final_tx["id"]

0        10003
1       100042
2        10012
3        10013
4        10014
         ...  
4508      9986
4509     99861
4510      9988
4511      9989
4512      9994
Name: id, Length: 4513, dtype: int64

In [18]:
columns = movies.iloc[:,[0,14,15,3]]
columns["id"]

0        10003
1       100042
2        10012
3        10013
4        10014
         ...  
4508      9986
4509     99861
4510      9988
4511      9989
4512      9994
Name: id, Length: 4513, dtype: int64

In [19]:
final = columns.merge(final_tx, how = "inner", on="id")
final

,id,release_date,revenue,budget,Year,Ticket Price
0,10003,1997-04-03,118063304,68000000,1997,4.59
1,100042,2014-11-12,169837010,40000000,2014,8.17
2,10012,2005-02-25,19294901,35000000,2005,6.41
3,10013,1986-10-05,41382841,18000000,1986,3.71
4,10014,1985-10-31,29999213,3000000,1985,3.55
...,...,...,...,...,...,...
4508,9986,2006-12-15,144000000,85000000,2006,6.55
4509,99861,2015-04-22,1405403694,280000000,2015,8.43
4510,9988,2006-08-25,19179969,17500000,2006,6.55
4511,9989,2001-01-12,18195610,30000000,2001,5.66


In [20]:
#final = np.concatenate((columns,final_tx.iloc[:,[1,2]]),axis=1)

In [21]:
final["ROI"] = (final["revenue"]-final["budget"])/(final["budget"])

In [22]:
final["Tickets Sold"] = final["revenue"]/final["Ticket Price"]

In [23]:
#final = final.drop(["Estimated Tickets Sold"],axis=1)

In [24]:
#Checking with Titanic & Tangled

In [25]:
titanic = movies[(movies["id"]==597)]

In [26]:
final[(final["id"] == 597)]

,id,release_date,revenue,budget,Year,Ticket Price,ROI,Tickets Sold
3415,597,1997-11-18,1845034188,200000000,1997,4.59,8.225171,4.019682e+08


In [27]:
despic_me = movies[(movies["title"]=="Despicable Me")]

In [28]:
final[(final["id"]==20352)]

,id,release_date,revenue,budget,Year,Ticket Price,ROI,Tickets Sold
1646,20352,2010-07-08,543513985,69000000,2010,7.89,6.877014,6.888644e+07


In [29]:
sound_of_music = movies[(movies["title"]=="The Sound of Music")]
sound_of_music

,id,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
1122,15121,False,NaN,8200000,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",NaN,tt0059742,en,The Sound of Music,Film adaptation of a classic Rodgers and Hamme...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The happiest sound in all the world!,The Sound of Music,False,7.4,966,"[{'cast_id': 1, 'character': 'Maria', 'credit_...","[{'credit_id': '52fe463b9251416c75071a61', 'de...","[{'id': 836, 'name': 'resistance'}, {'id': 120..."


In [30]:
final[(final["id"]==15121)]

,id,release_date,revenue,budget,Year,Ticket Price,ROI,Tickets Sold
1122,15121,1965-03-02,286214286,8200000,1965,1.01,33.904181,2.833805e+08


In [31]:
shrek_2 = movies[(movies["title"]=="Shrek 2")]
shrek_2

,id,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
3848,809,False,"{'id': 2150, 'name': 'Shrek Collection', 'post...",150000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",http://www.shrek2.com/,tt0298148,en,Shrek 2,"Shrek, Fiona and Donkey set off to Far, Far Aw...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Once upon another time...,Shrek 2,False,6.7,3082,"[{'cast_id': 29, 'character': 'Shrek (voice)',...","[{'credit_id': '57d85f1ac3a36878e90025c4', 'de...","[{'id': 378, 'name': 'prison'}, {'id': 2343, '..."


In [32]:
final[(final["id"]==809)]

,id,release_date,revenue,budget,Year,Ticket Price,ROI,Tickets Sold
3848,809,2004-05-19,919838758,150000000,2004,6.21,5.132258,1.481222e+08


In [33]:
final["Revenue in 2023"] = final["Tickets Sold"]*10.45

In [34]:
final

,id,release_date,revenue,budget,Year,Ticket Price,ROI,Tickets Sold,Revenue in 2023
0,10003,1997-04-03,118063304,68000000,1997,4.59,0.736225,2.572185e+07,2.687934e+08
1,100042,2014-11-12,169837010,40000000,2014,8.17,3.245925,2.078788e+07,2.172334e+08
2,10012,2005-02-25,19294901,35000000,2005,6.41,-0.448717,3.010125e+06,3.145581e+07
3,10013,1986-10-05,41382841,18000000,1986,3.71,1.299047,1.115440e+07,1.165635e+08
4,10014,1985-10-31,29999213,3000000,1985,3.55,8.999738,8.450483e+06,8.830754e+07
...,...,...,...,...,...,...,...,...,...
4508,9986,2006-12-15,144000000,85000000,2006,6.55,0.694118,2.198473e+07,2.297405e+08
4509,99861,2015-04-22,1405403694,280000000,2015,8.43,4.019299,1.667146e+08,1.742167e+09
4510,9988,2006-08-25,19179969,17500000,2006,6.55,0.095998,2.928240e+06,3.060010e+07
4511,9989,2001-01-12,18195610,30000000,2001,5.66,-0.393480,3.214772e+06,3.359437e+07


In [35]:
final

,id,release_date,revenue,budget,Year,Ticket Price,ROI,Tickets Sold,Revenue in 2023
0,10003,1997-04-03,118063304,68000000,1997,4.59,0.736225,2.572185e+07,2.687934e+08
1,100042,2014-11-12,169837010,40000000,2014,8.17,3.245925,2.078788e+07,2.172334e+08
2,10012,2005-02-25,19294901,35000000,2005,6.41,-0.448717,3.010125e+06,3.145581e+07
3,10013,1986-10-05,41382841,18000000,1986,3.71,1.299047,1.115440e+07,1.165635e+08
4,10014,1985-10-31,29999213,3000000,1985,3.55,8.999738,8.450483e+06,8.830754e+07
...,...,...,...,...,...,...,...,...,...
4508,9986,2006-12-15,144000000,85000000,2006,6.55,0.694118,2.198473e+07,2.297405e+08
4509,99861,2015-04-22,1405403694,280000000,2015,8.43,4.019299,1.667146e+08,1.742167e+09
4510,9988,2006-08-25,19179969,17500000,2006,6.55,0.095998,2.928240e+06,3.060010e+07
4511,9989,2001-01-12,18195610,30000000,2001,5.66,-0.393480,3.214772e+06,3.359437e+07


In [36]:
final_1 = movies.merge(final, how = "outer", on="id")

In [37]:
months = []
for i in range(len(release_dates_filt)):
    entry_month = dates.iloc[i][1].month
    months.append([release_dates_filt.iloc[i][0],entry_month])

months_arr = np.array(months)
months_arr = months_arr.T

months_df = pd.DataFrame()
months_df["id"] = months_arr[0]
months_df["Month"] = months_arr[1]

In [38]:
months_df

,id,Month
0,10003,4
1,100042,11
2,10012,2
3,10013,10
4,10014,10
...,...,...
4508,9986,12
4509,99861,4
4510,9988,8
4511,9989,1


In [39]:
final_2 = final_1.merge(months_df, how = "inner", on="id")
final_2

,id,adult,belongs_to_collection,budget_x,genres,homepage,imdb_id,original_language,original_title,overview,...,keywords,release_date_y,revenue_y,budget_y,Year,Ticket Price,ROI,Tickets Sold,Revenue in 2023,Month
0,10003,False,"{'id': 86224, 'name': 'The Saint Collection', ...",68000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,tt0120053,en,The Saint,"Ivan Tretiak, Russian Mafia boss who wants to ...",...,"[{'id': 220, 'name': 'berlin'}, {'id': 2139, '...",1997-04-03,118063304,68000000,1997,4.59,0.736225,2.572185e+07,2.687934e+08,4
1,100042,False,"{'id': 96665, 'name': 'Dumb and Dumber Collect...",40000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2096672,en,Dumb and Dumber To,20 years after the dimwits set out on their fi...,...,"[{'id': 6054, 'name': 'friendship'}, {'id': 96...",2014-11-12,169837010,40000000,2014,8.17,3.245925,2.078788e+07,2.172334e+08,11
2,10012,False,NaN,35000000,"[{'id': 27, 'name': 'Horror'}, {'id': 35, 'nam...",NaN,tt0257516,en,Cursed,A werewolf loose in Los Angeles changes the li...,...,"[{'id': 1155, 'name': 'brother sister relation...",2005-02-25,19294901,35000000,2005,6.41,-0.448717,3.010125e+06,3.145581e+07,2
3,10013,False,NaN,18000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0091738,en,Peggy Sue Got Married,Peggy Sue faints at a Highschool reunion. When...,...,"[{'id': 4379, 'name': 'time travel'}, {'id': 1...",1986-10-05,41382841,18000000,1986,3.71,1.299047,1.115440e+07,1.165635e+08,10
4,10014,False,"{'id': 8581, 'name': 'A Nightmare on Elm Stree...",3000000,"[{'id': 27, 'name': 'Horror'}]",NaN,tt0089686,en,A Nightmare on Elm Street Part 2: Freddy's Rev...,A new family moves into the house on Elm Stree...,...,"[{'id': 237, 'name': 'gay'}, {'id': 246, 'name...",1985-10-31,29999213,3000000,1985,3.55,8.999738,8.450483e+06,8.830754e+07,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4508,9986,False,NaN,85000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...",NaN,tt0413895,en,Charlotte's Web,Wilbur the pig is scared of the end of the sea...,...,"[{'id': 1701, 'name': 'hero'}, {'id': 3875, 'n...",2006-12-15,144000000,85000000,2006,6.55,0.694118,2.198473e+07,2.297405e+08,12
4509,99861,False,"{'id': 86311, 'name': 'The Avengers Collection...",280000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://marvel.com/movies/movie/193/avengers_ag...,tt2395427,en,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,...,"[{'id': 8828, 'name': 'marvel comic'}, {'id': ...",2015-04-22,1405403694,280000000,2015,8.43,4.019299,1.667146e+08,1.742167e+09,4
4510,9988,False,NaN,17500000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt0486551,en,Beerfest,During a trip to Germany to scatter their gran...,...,"[{'id': 74, 'name': 'germany'}, {'id': 271, 'n...",2006-08-25,19179969,17500000,2006,6.55,0.095998,2.928240e+06,3.060010e+07,8
4511,9989,False,NaN,30000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",http://www.antitrustthemovie.com/,tt0218817,en,Antitrust,A computer programmer's dream job at a hot Por...,...,"[{'id': 1576, 'name': 'technology'}, {'id': 21...",2001-01-12,18195610,30000000,2001,5.66,-0.393480,3.214772e+06,3.359437e+07,1


In [40]:
final_2.to_csv('movies_inflation_2.csv',index=False)

In [41]:
movies_infl = pd.read_csv("movies_inflation_2.csv")
movies_infl.iloc[0]

id                                                                   10003
adult                                                                False
belongs_to_collection    {'id': 86224, 'name': 'The Saint Collection', ...
budget_x                                                          68000000
genres                   [{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...
homepage                                                               NaN
imdb_id                                                          tt0120053
original_language                                                       en
original_title                                                   The Saint
overview                 Ivan Tretiak, Russian Mafia boss who wants to ...
popularity                                                        10.97633
poster_path                               /uA24D8JCg21RDINMZ3vFaQYdIzW.jpg
production_companies     [{'name': 'Paramount Pictures', 'id': 4}, {'na...
production_countries     